In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Project 3. Due by Sunday, March 3. ##

We will again use the **NYISO Data**.  In particular we will use the file "nyisobuses.csv".
<br>


Column **O** of this file is labeled "**Gen MW**". We call this value "generation".  Column **M** is "**Load MW**". We call this value "load."

For a bus i, **if generation(i) - load(i) > 0** we say that bus i is a **source**, and that its **maximum supply** equals **generation(i) - load(i)**.

For example, bus Number 3507 has generation = 182.77 and load = 52.59, and therefore it is a source and its maximum supply is 130.18.


And, for a bus i, if **generation(i) - load(i) < 0** then we say that bus i is a **destination**, and that its **maximum demand** equals **load(i) - generation(i)**.

For example, bus Number 3630 has generation = 0 and load = 28.68, and therefore it is a destination and its maximum demand is 28.68.



1. In this project we will compute the maximum amount of flow that can be sent from sources (in aggregate) to destinations (also in aggregate).  We will assume that each transmission line can be used in either direction, and that its capacity will be a number **C **-- **the same value C for all transmission lines**.  This parameter will be an input to the problem.

2. Write a python script that sets up and solves the above problem as a maximum flow problem, by adding a supersource connected to each source (with capacity equal to the maximum supply of that source) and a superdestination connected from each destination (with capacity equal to the maximum demand at that destination). Note it is the original transmission lines that have capacity C.  The user inputs this value on the command line.

3. You are free to use my code for setting up a max flow problem into one that can be solved with Gurobi.

4. When you run your code, it should identify those lines that are at capacity and produce a list.

5. **Extra credit**. After each run, you should perform a flow decomposition as discussed in class.  Try to produce routes that are as short as possible. (Heuristics are needed!)


## Import Packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 27.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import random
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
from geopy.distance import great_circle, geodesic
import matplotlib.pyplot as plt
from copy import deepcopy

## Data Preprocessing

### Data Analysis

In [4]:
data_dir = '/content/drive/My Drive/Spring_2024/IEOR4004/Data/NewYorkElectricGrid/'
buses = pd.read_csv(data_dir+'nyisobuses.csv', skiprows=1, index_col=[0])
branches = pd.read_csv(data_dir+'nyisobranches.csv')

In [5]:
print(buses.columns)

Index(['Name', 'Area Name', 'Latitude', 'Longitude', 'Status',
       '# Neighbors (in service only)', 'Nom kV', 'PU Volt', 'Volt (kV)',
       'Angle (Deg)', 'Load MVA', 'Load MW', 'Load Mvar', 'Gen MW', 'Gen Mvar',
       'Gen Min MW', 'Gen Max MW', 'Gen Min Mvar', 'Gen Max Mvar',
       'Switched Shunts Mvar', 'Switched Shunts Nom Mvar', 'Max Mvar',
       'Min Mvar', 'Area Num', 'Zone Num'],
      dtype='object')


In [6]:
dscb = buses[['Load MW', 'Gen MW']].describe()
dscb.loc['NA count'] = buses[['Load MW', 'Gen MW']].isna().sum()
dscb

,Load MW,Gen MW
count,1611.000000,118.00000
mean,16.854351,232.44678
std,22.041125,320.92010
min,-5.010000,5.87000
25%,4.455000,38.97000
50%,9.990000,85.25500
75%,22.050000,314.46000
max,388.530000,1901.10000
NA count,203.000000,1696.00000


In [7]:
nan_indices_buses = buses.index[buses['Gen MW'].isna() & buses['Load MW'].isna()].to_list()
duplicate_indices_buses = buses.index[buses.duplicated(subset = ['Name'], keep = False)].to_list()
print(buses.loc[nan_indices_buses, 'Name'])
print(buses.loc[duplicate_indices_buses, 'Name'])

Number
123                          J C McNeil
187                          9001043100
239                          9001071000
338                          9001245500
845                          9011690300
                      ...              
77627    PSEG Linden Generating Station
77635         Bergen Generating Station
77640                Linden Cogen Plant
78955                       42069110600
95339                         PJM Proxy
Name: Name, Length: 175, dtype: object
Number
3362                        50007002200
3502                     Indian Point 2
3503                     Indian Point 3
3504                Oswego Harbor Power
3505                    Charles Poletti
                      ...              
77626    PSEG Linden Generating Station
77627    PSEG Linden Generating Station
77635         Bergen Generating Station
78036                       34017010800
78955                       42069110600
Name: Name, Length: 209, dtype: object


In [8]:
print(buses.loc[[3505, 76457]])

                   Name Area Name  Latitude  Longitude     Status  \
Number                                                              
3505    Charles Poletti  New York    40.776    -73.911  Connected   
76457   Charles Poletti  New York    40.775    -73.910  Connected   

        # Neighbors (in service only)  Nom kV  PU Volt  Volt (kV)  \
Number                                                              
3505                                2   230.0  0.99996    229.990   
76457                               8   138.0  0.98440    135.848   

        Angle (Deg)  ...  Gen Min MW  Gen Max MW  Gen Min Mvar  Gen Max Mvar  \
Number               ...                                                       
3505        -123.03  ...         1.0       883.0           0.0           0.0   
76457       -125.33  ...         NaN         NaN           NaN           NaN   

        Switched Shunts Mvar  Switched Shunts Nom Mvar  Max Mvar  Min Mvar  \
Number                                         

In [ ]:
dup_dict_buses = {}
not_first_dup_buses = {}
for index in duplicate_indices_buses:
    name = buses.loc[index, 'Name']
    if name not in dup_dict_buses.keys():
        dup_dict_buses[name] = [index]
    else:
        dup_dict_buses[name].append(index)
        not_first_dup_buses[index] = dup_dict_buses[name][0]

In [ ]:
for name in dup_dict_buses.keys():
    print(buses.loc[dup_dict_buses[name], ['Gen MW', 'Load MW']])
    dup_list = dup_dict_buses[name]
    buses.loc[dup_list[0], 'Gen MW'] = buses.loc[dup_list, 'Gen MW'].replace(0.0, np.nan).mean()
    buses.loc[dup_list[0], 'Load MW'] = buses.loc[dup_list, 'Load MW'].replace(0.0, np.nan).mean()
    print(buses.loc[dup_dict_buses[name], ['Gen MW', 'Load MW']])
    print('*******************************************')

        Gen MW  Load MW
Number                 
3362       NaN      NaN
76428      NaN      NaN
        Gen MW  Load MW
Number                 
3362       NaN      NaN
76428      NaN      NaN
*******************************************
         Gen MW  Load MW
Number                  
3502    1138.53      0.0
76453       NaN      NaN
         Gen MW  Load MW
Number                  
3502    1138.53      NaN
76453       NaN      NaN
*******************************************
        Gen MW  Load MW
Number                 
3503    1012.0      0.0
76454      NaN      NaN
76455      NaN      NaN
        Gen MW  Load MW
Number                 
3503    1012.0      NaN
76454      NaN      NaN
76455      NaN      NaN
*******************************************
        Gen MW  Load MW
Number                 
3504    376.93      0.0
76456      NaN      NaN
        Gen MW  Load MW
Number                 
3504    376.93      NaN
76456      NaN      NaN
*******************************************


In [ ]:
branches.describe()

,branch number,first bus number,second bus number
count,2203.000000,2203.000000,2203.000000
mean,1102.000000,14596.378121,14601.174762
std,636.095643,23600.647157,22852.389163
min,1.000000,123.000000,3503.000000
25%,551.500000,4259.000000,5209.500000
50%,1102.000000,6190.000000,6697.000000
75%,1652.500000,7648.500000,8054.000000
max,2203.000000,95342.000000,95341.000000


In [ ]:
for index in branches.index:
    b1 = branches.loc[index, ' first bus number'].astype('int64')
    b2 = branches.loc[index, ' second bus number'].astype('int64')
    if b1 in not_first_dup_buses.keys():
        branches.loc[index, ' first bus number'] = not_first_dup_buses[b1]
    if b2 in not_first_dup_buses.keys():
        branches.loc[index, ' second bus number'] = not_first_dup_buses[b2]
    # if (b1 in nan_indices_buses) or (b2 in nan_indices_buses):
    #     branches.drop(index = index, inplace = True)
branches.drop_duplicates(subset = [' first bus number', ' second bus number'], keep = 'first', inplace = True)
branches.describe()

,branch number,first bus number,second bus number
count,2142.000000,2142.000000,2142.000000
mean,1087.942577,5777.099440,6518.701681
std,627.896608,3855.729268,4666.144046
min,1.000000,123.000000,3502.000000
25%,544.250000,3944.250000,4956.250000
50%,1083.500000,5743.500000,6426.000000
75%,1632.750000,6985.000000,7552.250000
max,2186.000000,95342.000000,95341.000000


In [ ]:
print(len(set(branches[' first bus number'].to_list() + branches[' second bus number'].to_list())))
print(len(buses))

1701
1701


In [ ]:
a = set(branches[[' first bus number', ' second bus number']].values.flatten().tolist())
b = set(buses.index.to_list())
a == b

True

### Preprocessing Function

In [ ]:
def dataReader(data_dir):
    # Read Data from csv files
    buses = pd.read_csv(data_dir+'nyisobuses.csv', skiprows=1, index_col=[0])
    branches = pd.read_csv(data_dir+'nyisobranches.csv')
    return buses, branches

In [ ]:
def dataPreprocess(buses_original, branches_original):
    # Deepcopy the dfs
    buses = buses_original.copy(deep = True)
    branches = branches_original.copy(deep = True)

    """BUS"""
    # Get nan_indices_buses and duplicate_indices_buses
    # nan_indices_buses: the indices in buses where 'Gen MW' and 'Load MW' are both nan's
    nan_indices_buses = buses.index[buses['Gen MW'].isna() & buses['Load MW'].isna()].to_list()
    # duplicate_indices_buses: the indices in buses where 'Name' are duplicates
    duplicate_indices_buses = buses.index[buses.duplicated(subset = ['Name'], keep = False)].to_list()

    # # *************** NOT SURE IF WE SHOULD DO THE FOLLOWING STEP **************
    # # Drop the rows where 'Gen MW' and 'Load MW' are both nan's
    # buses.dropna(index = nan_indices_buses, inplace = True)

    # Get the Dictionary dup_dict_buses and not_first_dup_buses
    # dup_dict_buses:
    #     keys: the names of the duplicates bus
    #     values: list of indices sharing the same name, numerially ordered
    # not_first_dup_buses
    #     keys: the indices of buses after their respective first appearance
    #     values: the corresponding first appearance
    dup_dict_buses = {}
    not_first_dup_buses = {}
    for index in duplicate_indices_buses:
        name = buses.loc[index, 'Name']
        if name not in dup_dict_buses.keys():
            dup_dict_buses[name] = [index]
        else:
            dup_dict_buses[name].append(index)
            not_first_dup_buses[index] = dup_dict_buses[name][0]

    # For each duplicated bus, modify the 'Gen MW' and 'Load MW' of the first appearence
    # Fill in mean while omitting the nan and 0.0; if all are nan's then fill in 0.0
    for name in dup_dict_buses.keys():
        dup_list = dup_dict_buses[name]
        gen_mw = buses.loc[dup_list, 'Gen MW'].replace(0.0, np.nan).mean(skipna = True)
        buses.loc[dup_list[0], 'Gen MW'] = gen_mw if gen_mw != np.nan else 0.0
        load_mw = buses.loc[dup_list, 'Load MW'].replace(0.0, np.nan).mean(skipna = True)
        buses.loc[dup_list[0], 'Load MW'] = load_mw if load_mw != np.nan else 0.0

    # Drop all duplicates except the first appearance in buses
    buses.drop(index = list(not_first_dup_buses.keys()), inplace = True)

    # Fill 0.0 in all nans under 'Gen MW' and 'Load MW'
    buses.fillna({'Gen MW': 0.0, 'Load MW': 0.0}, inplace = True)

    """BRANCHES"""
    # During exploratory analysis, no nan is found in branches

    # For each branch, check if any of the bus numbers are duplicates found previously
    # If so, replace them with the indices of their first appearance
    for index in branches.index:
        b1 = branches.loc[index, ' first bus number'].astype('int64')
        b2 = branches.loc[index, ' second bus number'].astype('int64')
        if b1 in not_first_dup_buses.keys():
            branches.loc[index, ' first bus number'] = not_first_dup_buses[b1]
        if b2 in not_first_dup_buses.keys():
            branches.loc[index, ' second bus number'] = not_first_dup_buses[b2]
        # # *************** NOT SURE IF WE SHOULD DO THE FOLLOWING STEP **************
        # if (b1 in nan_indices_buses) or (b2 in nan_indices_buses):
        #     branches.drop(index = index, inplace = True)

    # Drop the duplicates in branches
    branches.drop_duplicates(subset = [' first bus number', ' second bus number'], keep = 'first', inplace = True)

    """Completion Check"""
    # Check every bus appearing in branches is in buses
    # & Check every bus in buses is in branches
    a = set(branches[[' first bus number', ' second bus number']].values.flatten().tolist())
    b = set(buses.index.to_list())
    assert a == b, "buses in Dataframe(buses) and buses in Dataframe(branches) don't match."

    return buses, branches

## Data Loader

In [ ]:
def dataLoader(data_dir, c):
    print('----Reading network files from' + data_dir + '----\n')
    buses_orig, branches_orig = dataReader(data_dir)
    print('----Preprocessing network files----\n')
    buses, branches = dataPreprocess(buses_orig, branches_orig)

    print('----Generating graph----\n')
    G = nx.DiGraph()
    source_nodes = []
    target_nodes = []
    G.add_node('_s', balance = 0)
    G.add_node('_t', balance = 0)
    for index in tqdm(branches.index):
        b1 = branches.loc[index, ' first bus number'].astype('int64')
        b2 = branches.loc[index, ' second bus number'].astype('int64')
        f1 = buses.loc[b1, 'Gen MW'] - buses.loc[b1, 'Load MW']
        f2 = buses.loc[b2, 'Gen MW'] - buses.loc[b2, 'Load MW']
        G.add_node(b1, balance = 0)
        G.add_node(b2, balance = 0)
        G.add_edge(b1, b2, capacity = c)
        G.add_edge(b2, b1, capacity = c)
        if (f1 > 0) and (b1 not in source_nodes):
            source_nodes.append(b1)
            G.add_edge('_s', b1, capacity = f1)
        elif (f1 < 0) and (b1 not in target_nodes):
            target_nodes.append(b1)
            G.add_edge(b1, '_t', capacity = -f1)
        if (f2 > 0) and (b2 not in source_nodes):
            source_nodes.append(b2)
            G.add_edge('_s', b2, capacity = f2)
        elif (f2 < 0) and (b2 not in target_nodes):
            target_nodes.append(b2)
            G.add_edge(b2, '_t', capacity = -f2)
    print('----Successfully generated graph----\n')
    return G, source_nodes, target_nodes

## MaxFlow

In [ ]:
class MaxFlow():
    def __init__(self, capacity):
        self.data_dir = '/content/drive/My Drive/Spring_2024/IEOR4004/Data/NewYorkElectricGrid/'
        self.capacity = capacity
        self.graph, self.source_nodes, self.target_nodes = DataLoader.dataLoader(self.data_dir, self.capacity)
        self.model, self.flowvar, self.flowamountvar = self.lpcreator('NewYorkElectricGrid')

    def lpcreator(self, name):
        print(f'----Creating model {name} with transmission capacity {self.capacity}----\n')
        m = gp.Model(name)
        flowvar = {}

        print(f'**** Adding Variables ****\n')
        for u, v, cap in tqdm(self.graph.edges.data('capacity')):
            flowvar[(u,v)] = m.addVar(name = f'f{u},{v}', ub = cap)
        flowamountvar = m.addVar(name = 'flow')

        print(f'**** Setting Objective ****\n')
        m.setObjective(flowamountvar, GRB.MAXIMIZE)

        print(f'**** Adding Constraints ****\n')
        for node in self.graph.nodes:
            expr = gp.LinExpr()
            for successor in self.graph.successors(node):
                expr += flowvar[(node, successor)]
            for predecessor in self.graph.predecessors(node):
                expr -= flowvar[(predecessor, node)]
            if node == '_s':
                m.addConstr(expr - flowamountvar == 0, name = f'Balance{node}')
            elif node == '_t':
                m.addConstr(expr + flowamountvar == 0, name = f'Balance{node}')
            else:
                m.addConstr(expr == 0, name = f'Balance{node}')
        m.update()
        m.write(f'{name}_{self.capacity}.lp')
        print(f'----Successfully created model {name} with transmission capacity {self.capacity}----\n')
        m.optimize()
        print(f'----Successfully optimized model {name} with transmission capacity {self.capacity}----\n')
        return m, flowvar, flowamountvar

    def get_branches_at_capacity(self):
        lst = []
        for u, v, cap in tqdm(self.graph.edges.data('capacity')):
          if abs(self.flowvar[(u, v)].x - cap) < 1e-6:
              lst.append((u, v))
              # print(f'The branch {(u, v)} is at capacity')
        return lst

    def flow_decomposition(self):
        G = deepcopy(self.graph)
        for u, v in G.edges:
            flow = self.flowvar[(u, v)].x
            if flow <= 1e-6:
                G.remove_edge(u, v)
            else:
                G[u][v]['flow'] = flow

        s = '_s'
        t = '_t'
        total = self.flowamountvar.x
        paths = []
        print('\n*** Flow Decomposition***\n')
        while total > 1e-6:
            path = [s]
            path_flow = float('inf')
            while path[-1] != t:
                curr_node = path[-1]
                next_nodes = [v for v in G.successors(curr_node) if G[curr_node][v]['flow'] > 1e-6]
                next_node = min(next_nodes, key = lambda x: nx.shortest_path_length(G, x, t))
                path.append(next_node)
                path_flow = min(path_flow, G[curr_node][next_node]['flow'])

            total -= path_flow
            paths.append((path, path_flow))
            print(f'path: {path}, path flow: {path_flow}\n')
            for i in range(1, len(path)):
                G[path[i-1]][path[i]]['flow'] -= path_flow
                if G[path[i-1]][path[i]]['flow'] <= 1e-6:
                    G.remove_edge(path[i-1], path[i])

        return paths